In [1]:
import ray
#ray.init()

In [2]:
def create_model():
    import tensorflow as tf
    print(tf.config.list_physical_devices('GPU'))
    
    @tf.function
    def model(x):
        return -1*(x*x)
    
    def grad(self,x):
        x  = tf.convert_to_tensor(x)
        with tf.GradientTape() as tape:
            tape.watch(x)
            res = model(x)
        return tape.gradient(res, [x])
    
    return model, grad

In [3]:
#ray.init()


In [4]:
@ray.remote(num_gpus=1)
class Simulator(object):
    def __init__(self):
        self.model, self.grad = create_model()
        
    def fwd(self,x):
        return self.model(x)
    
    def gradient(self,x):
        return self.grad(x)

In [5]:
my_sim=Simulator

In [6]:
my_sim.gradient(0.1)

AttributeError: 'ActorClass(Simulator)' object has no attribute 'grad'

In [7]:
my_sim.fwd(0.1)

AttributeError: 'ActorClass(Simulator)' object has no attribute 'model'

In [8]:
simulators = [Simulator.remote() for _ in range(2)]


RaySystemError: System error: Ray has not been started yet. You can start Ray with 'ray.init()'.

In [ ]:
from ray.util.multiprocessing import Pool as RayPool
import bayesfast as bf
bf.utils.parallel.set_backend(RayPool(4))
bf.utils.random.set_generator(4)
den = bf.DensityLite(logp=my_sim.model,grad=my_sim.fwd,input_size=1)
#

2020-10-20 13:05:01,926	WARNING worker.py:1072 -- The actor or task with ID ffffffffffffffff62223d8501000000 is pending and cannot currently be scheduled. It requires {CPU: 1.000000} for execution and {CPU: 1.000000} for placement, but this node only has remaining {node:128.55.224.191: 1.000000}, {accelerator_type:V100: 1.000000}, {GPU: 1.000000}, {memory: 232.324219 GiB}, {object_store_memory: 71.435547 GiB}. In total there are 0 pending tasks and 4 pending actors on this node. This is likely due to all cluster resources being claimed by actors. To resolve the issue, consider creating fewer actors or increase the resources available to this Ray cluster. You can ignore this message if this Ray cluster is expected to auto-scale.
2020-10-20 13:05:02,139	INFO (unknown file):0 -- gc.collect() freed 27 refs in 0.09156973997596651 seconds
(pid=71616) 2020-10-20 13:05:02,080	INFO (unknown file):0 -- gc.collect() freed 15 refs in 0.012529953964985907 seconds
(pid=71614) 2020-10-20 13:05:02,092

In [10]:
foo = bf.sample(den)

NameError: name 'den' is not defined

In [ ]:
simulators = [Network.remote() for _ in range(2)]